In [1]:
import sys, os
if 'google.colab' in sys.modules:
    %cd
    % rm -rf MPyDATA
    ! git clone --recurse-submodules -j8 https://github.com/Michaeldz36/MPyDATA.git
    %cd MPyDATA
    ! git checkout develop
    ! pip install -U $(cat requirements.txt | cut -d '=' -f 1)
else:
    sys.path.append(os.path.join(os.getcwd(), '../..'))

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from MPyDATA_examples.Olesik_et_al_2020.setup import Setup, default_opt_set
from MPyDATA_examples.Olesik_et_al_2020.coordinates import x_id, x_p2, x_p3, x_log_of_pn
from MPyDATA.options import Options
from MPyDATA_examples.Olesik_et_al_2020.simulation import Simulation
from MPyDATA_examples.utils.show_plot import show_plot
from joblib import Parallel, parallel_backend, delayed
from MPyDATA_examples.Olesik_et_al_2020.physics.equilibrium_drop_growth import PdfEvolver
from MPyDATA.arakawa_c.discretisation import discretised_analytical_solution
from MPyDATA_examples.utils.error_norms import L2, Smolarkiewicz_Grabowski_1990_eq21, modified_Smolarkiewicz_Rasch_r0
from MPyDATA_examples.Olesik_et_al_2020.convergence_plotter  import polar_plot

In [3]:
GCs = np.linspace(.05,.95, 2)
nrs = np.array([2**n for n in range(10, 12)], dtype=int)

In [4]:
grid_layout = x_p2()
psi_coord = x_p2()
setup = Setup()

In [5]:
def analysis(setup, GC, opt):
    options = Options(**opt)
    simulation = Simulation(setup, grid_layout,  psi_coord, options, GC)
    simulation.step(simulation.out_steps[-1])
    t = simulation.out_steps[-1] * simulation.dt
    rh = simulation.rh
    pdf_t = PdfEvolver(setup.pdf, setup.drdt, t)
    def pdf_arg(r):
        return pdf_t(r* rh.units).magnitude
    analytical = discretised_analytical_solution(
                simulation.rh.magnitude,
                pdf_arg, midpoint_value = True
            ) * pdf_t(rh[0]).units
    numerical = simulation.n_of_r
    psi = simulation.psi
    g_factor = simulation.g_factor
    dp_dr = simulation.dp_dr

    
    maximum_numeric = np.max(numerical)
    maximum_analytic = np.max(analytical)
    measure_height = (maximum_numeric / maximum_analytic).magnitude
    
    dif = analytical-numerical
    measure_h_2 =  (1/t * np.sqrt( 1/len(dif) * dif.dot(dif))).magnitude
        
    def rel_disp(r, psi, g_factor):
        #TODO: move to analysis/simulation and write over simulation plots
        mom0 = 0
        mom1 = 0
        mom2 = 0
        for i in range(len(psi)):
            psi_i = psi[i] 
            dp_i = psi_coord.moment_of_r_integral(psi_coord.x(r[i+1]),0) - psi_coord.moment_of_r_integral(psi_coord.x(r[i]),0)
            A_i = psi_coord.moment_of_r_integral(psi_coord.x(r[i+1]),1) - psi_coord.moment_of_r_integral(psi_coord.x(r[i]),1)
            B_i = psi_coord.moment_of_r_integral(psi_coord.x(r[i+1]),2) - psi_coord.moment_of_r_integral(psi_coord.x(r[i]),2)
            bin_mom0= psi_i * dp_i
            bin_mom1 = psi_i * A_i
            bin_mom2 = psi_i * B_i
            mom0+=bin_mom0
            mom1+=bin_mom1
            mom2+=bin_mom2
        mu = mom1 / mom0
        std = np.sqrt(mom2/mom0 - (mu)**2)
        d = std/mu
        return d    
    numeric_rel_d = rel_disp(rh, numerical/dp_dr, g_factor)
    analytic_rel_d = rel_disp(rh, analytical/dp_dr, g_factor)
    print("analytic rd", analytic_rel_d)
    print("numeric rd", numeric_rel_d)
    dispersion_ratio = numeric_rel_d / analytic_rel_d
    print("dispersion ratio", dispersion_ratio)
    print("\n")

    error = np.log2(Smolarkiewicz_Grabowski_1990_eq21(numerical.magnitude, analytical.magnitude, t.magnitude))
    error_g = np.log2(Smolarkiewicz_Grabowski_1990_eq21(g_factor * psi.magnitude, g_factor * analytical.magnitude / dp_dr, t.magnitude))
    error_r0 = np.log2(modified_Smolarkiewicz_Rasch_r0(psi.magnitude, analytical.magnitude /dp_dr, t.magnitude, g_factor))
    
    return setup.nr, GC, error, error_g, error_r0, measure_height, measure_h_2

In [6]:
opts = default_opt_set
def replace_names(opt):
    str_repl = [["'n_iters': 1","upwind"],
                ["'n_iters': 2","MPDATA 2 iterations"],
                ["'n_iters': 3","MPDATA 3 iterations"],
                ["'",""],
                [": True",""],
                ["_", " "],
                ["{",""],["}",""],[","," "]]                            
    for repl in str_repl:
        opt = str(opt).replace(repl[0], repl[1])
    return opt

In [ ]:
for opt in opts:
    with parallel_backend('threading', n_jobs=-2):
        results0 = Parallel(verbose=10)(
            delayed(analysis)(Setup(nr = nr, mixing_ratios_g_kg = [1.05,]), GC, opt)
            for nr in nrs
            for GC in GCs
        )
    results = tuple(tuple(i) for i in zip(*results0))
    plot_setup = np.array(results[0:2])
    measures = {'log$_2$(err)':results[2] , 'log$_2$(err_g)':results[3] , 'log$_2$(err_r0)':results[4], 'height difference':results[5], 'h_2': results[6]}
    opt = replace_names(opt)
    print(opt)
    for measure in measures.keys():
        polar_plot(nrs, GCs, measures[measure], name = measure)
        show_plot(filename = f'convergence_{measure}_{opt}.pdf')   #TODO: dont show plot name when savingfig (for masters)

[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.


analytic rd 0.33905495608084285 dimensionless
numeric rd 0.33923974435962356 dimensionless
dispersion ratio 1.0005450098146822 dimensionless


analytic rd 0.3396087628473136 dimensionless
numeric rd 0.33970721759729633 dimensionless
dispersion ratio 1.0002899063886257 dimensionless


analytic rd 0.3394239087225427 dimensionless
numeric rd 0.33968637009278685 dimensionless
dispersion ratio 1.0007732553998094 dimensionless


analytic rd 0.33979386972283987 dimensionless
numeric rd 0.34013006087753955 dimensionless
dispersion ratio 1.0009893973513233 dimensionless




[Parallel(n_jobs=-2)]: Done   4 tasks      | elapsed:  2.3min


analytic rd 0.3381369516142046 dimensionless
numeric rd 0.33815461286260456 dimensionless
dispersion ratio 1.0000522310511042 dimensionless


analytic rd 0.3394273223708338 dimensionless
numeric rd 0.33955924779696167 dimensionless
dispersion ratio 1.000388670614983 dimensionless


analytic rd 0.33979734211502066 dimensionless
numeric rd 0.3399665968204824 dimensionless
dispersion ratio 1.0004981048539352 dimensionless


analytic rd 0.3390583119639061 dimensionless
numeric rd 0.3391511163893536 dimensionless
dispersion ratio 1.000273712285388 dimensionless


analytic rd 0.3396122057432932 dimensionless
numeric rd 0.3396617233619479 dimensionless
dispersion ratio 1.0001458063574196 dimensionless


analytic rd 0.33814016720523987 dimensionless
numeric rd 0.33814945375746097 dimensionless
dispersion ratio 1.0000274636175224 dimensionless




[Parallel(n_jobs=-2)]: Done  10 out of  20 | elapsed:  4.6min remaining:  4.6min


...

[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.


analytic rd 0.33979386972283987 dimensionless
numeric rd 0.3397934643657182 dimensionless
dispersion ratio 0.9999988070499272 dimensionless


analytic rd 0.3381369516142046 dimensionless
numeric rd 0.3381353894660694 dimensionless
dispersion ratio 0.9999953801318439 dimensionless


analytic rd 0.3396087628473136 dimensionless
numeric rd 0.3396091088035234 dimensionless
dispersion ratio 1.0000010186904686 dimensionless


analytic rd 0.3394239087225427 dimensionless
numeric rd 0.3394248864057933 dimensionless
dispersion ratio 1.0000028804195153 dimensionless




[Parallel(n_jobs=-2)]: Done   4 tasks      | elapsed:  2.8min


analytic rd 0.33905495608084285 dimensionless
numeric rd 0.3390561141746069 dimensionless
dispersion ratio 1.0000034156520743 dimensionless


analytic rd 0.33979734211502066 dimensionless
numeric rd 0.3397966442123946 dimensionless
dispersion ratio 0.9999979461209975 dimensionless


analytic rd 0.3394273223708338 dimensionless
numeric rd 0.33942708642052916 dimensionless
dispersion ratio 0.9999993048576556 dimensionless


analytic rd 0.3390583119639061 dimensionless
numeric rd 0.3390582612216516 dimensionless
dispersion ratio 0.999999850343576 dimensionless


analytic rd 0.3396122057432932 dimensionless
numeric rd 0.3396121092341999 dimensionless
dispersion ratio 0.9999997158256043 dimensionless


analytic rd 0.33814016720523987 dimensionless
numeric rd 0.33813973661443253 dimensionless
dispersion ratio 0.9999987265907777 dimensionless




[Parallel(n_jobs=-2)]: Done  10 out of  20 | elapsed:  5.7min remaining:  5.7min
